# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [9]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [10]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Analysis Techniques for Diffusion MRI	Talk	talk-4	ISMRM Workshop on 40 Years of Diffusion	16/02/2025	"Kyoto,  Japan"	https://github.com/ethompson93/dmri_analysis_techniques	Led educational bootcamp session on analysis techniques for diffusion MRI.
Structural Connectomics Informed by Large Language Models	Poster	poster-6	ISMRM Workshop on 40 Years of Diffusion	19/02/2025	"Kyoto,  Japan"		
Combining multimodal connectivity information improves modelling of pathology spread in Alzheimer's disease	Talk	talk-3	Alzheimer's Association International Conference (AAIC)	31/07/2024	"Philadelphia, USA"		Featured research session - Tau accumulation and tau spreading patterns II
Demonstration of an open-source toolbox for network spreading models: regional amyloid burden promotes tau production in Alzheimer's disease	Poster	poster-5	Alzheimer's Association International Conference (AAIC)	29/07/2024	"Philadelphia, USA"	http://ethompson93.gi

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [11]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Analysis Techniques for Diffusion MRI,Talk,talk-4,ISMRM Workshop on 40 Years of Diffusion,16/02/2025,"Kyoto, Japan",https://github.com/ethompson93/dmri_analysis_t...,Led educational bootcamp session on analysis t...
1,Structural Connectomics Informed by Large Lang...,Poster,poster-6,ISMRM Workshop on 40 Years of Diffusion,19/02/2025,"Kyoto, Japan",NaN,NaN
2,Combining multimodal connectivity information ...,Talk,talk-3,Alzheimer's Association International Conferen...,31/07/2024,"Philadelphia, USA",NaN,Featured research session - Tau accumulation a...
3,Demonstration of an open-source toolbox for ne...,Poster,poster-5,Alzheimer's Association International Conferen...,29/07/2024,"Philadelphia, USA",http://ethompson93.github.io/files/toolbox_pos...,NaN
4,Combining Multimodal Connectivity Improves Mod...,Poster,poster-4,OHBM Annual Meeting,23/07/2023,"Montreal, Canada",http://ethompson93.github.io/files/OHBM_2023.pdf,NaN
5,Data-Driven Decomposition of Diffusion MRI for...,Talk,talk-2,Neuroimaging in Health and Disease Seminar Series,19/01/2022,"Mallinckrodt Institute of Radiology, Washingto...",NaN,NaN
6,Non-Negative Decomposition of Structural Conne...,Virtual Poster,poster-3,OHBM Annual Meeting,01/06/2020,Online,http://ethompson93.github.io/files/OHBM_2020.pdf,NaN
7,Exploring the Gyral Bias on White Matter Tract...,Poster,poster-1,OHBM Annual Meeting,01/06/2019,"Rome, Italy",http://ethompson93.github.io/files/poster_gb.pdf,NaN
8,Data-Driven Mapping of Structural Connectivity...,Poster,poster-2,OHBM Annual Meeting,01/06/2019,"Rome, Italy",http://ethompson93.github.io/files/poster_ICA.pdf,NaN
9,Assessing the Asynchronous Macrostructural Cha...,Power Pitch,talk-1,ISMRM Annual Meeting,01/06/2018,"Paris, France",https://index.mirasmart.com/ISMRM2018/PDFfiles...,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [12]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [13]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

In [14]:
item.url_slug

'talk-1'

These files are in the talks directory, one directory below where we're working from.

In [15]:
!ls ../_talks

2018-talk-1.md   2020-poster-3.md 2024-poster-5.md 2025-talk-4.md
2019-poster-1.md 2022-talk-2.md   2024-talk-3.md
2019-poster-2.md 2023-poster-4.md 2025-poster-6.md


In [16]:
!cat ../_talks/2013-03-01-tutorial-1.md

cat: ../_talks/2013-03-01-tutorial-1.md: No such file or directory
